In [3]:
import libsbml
import os

from collections import defaultdict
from pprint import pprint

In [8]:
doc = libsbml.readSBMLFromFile("iPAE1146_with_groups.xml")
model = doc.getModel()

groups_plugin = model.getPlugin("groups")
groups = groups_plugin.getListOfGroups()

subsystem_balance = dict()
neighbors = dict()
for group in groups:
    net_balance = defaultdict(int)
    reactant_balance = defaultdict(int)
    product_balance = defaultdict(int)
    for member in group.getListOfMembers():
        r_id = member.getIdRef()
        reaction = model.getReaction(r_id) 
        for reactant in reaction.getListOfReactants():
            s_id = reactant.getSpecies()
            name = model.getSpecies(s_id).getName() 
            stoichiometry = reactant.getStoichiometry()
            reactant_balance[name] += stoichiometry
        for product in reaction.getListOfProducts():
            s_id = product.getSpecies()
            name = model.getSpecies(s_id).getName()
            stoichiometry = product.getStoichiometry()
            product_balance[name] += stoichiometry
        keys = set(product_balance.keys()).union(reactant_balance.keys())
        for key in keys:
            net_balance[key] += product_balance[key] - reactant_balance[key]
        to_remove = list()
        for key in net_balance.keys():
            if net_balance[key] == 0:
                to_remove.append(key)
        for key in to_remove:
            net_balance.pop(key)
    subsystem_balance[group.name] = net_balance

subsystem_metabolites = dict()
for key in subsystem_balance.keys():
    subsystem_metabolites[key] = list(subsystem_balance[key].keys())

    
candidates = defaultdict(dict)
chosen_groups = ["Lipopolysaccharide biosynthesis", 
                  "Amino sugar and nucleotide sugar metabolism", 
                  "Pyrimidine metabolism"]
for this in chosen_groups:
    for other in subsystem_metabolites.keys():
        if this == other:
            continue
        metabolites = set(subsystem_metabolites[this]).intersection(subsystem_metabolites[other])
        desc = this + " - " + other
        if metabolites:
            for metabolite in metabolites:
                is_part_sink = subsystem_balance[this][metabolite] > 0 and subsystem_balance[other][metabolite] < 0
                is_part_demand = subsystem_balance[this][metabolite] < 0 and subsystem_balance[other][metabolite] > 0
                if is_part_sink or is_part_demand:
                    candidates[desc][metabolite] = [subsystem_balance[this][metabolite], subsystem_balance[other][metabolite]]
                    
with open("Potential_boundary_metabolites.txt", "w") as file:
    pprint(candidates, file)

net_fluxes = defaultdict(dict)
for group in groups_plugin.getListOfGroups():
    if group.getName() not in chosen_groups:
        continue
    for member in group.getListOfMembers():
        r_id = member.getIdRef()
        reaction = model.getReaction(r_id)
        reversible = reaction.getReversible()
        sum_reactants = 0
        sum_products = 0
        for reactant in reaction.getListOfReactants():
            sum_reactants += reactant.getStoichiometry()
        for product in reaction.getListOfProducts():
            sum_products += product.getStoichiometry()
        net = sum_products - sum_reactants
        if net != 0:
            net_fluxes[group.getName()][reaction.getName()] = [net, reversible]
    
with open("Reaction_fluxes.txt", "w") as file:
    pprint(net_fluxes, file)